# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from numpy import expand_dims
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Activation,Dropout

# Loading Dataset

In [2]:
df = pd.read_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\FYP-I\\Master\\Data Collection\\CollectedData\\Final_Data (2022).csv", index_col = False)

In [3]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [4]:
df.head(7)

,Province,Country_Region,Date,Day,Month,Year,Lat,Long_,Confirmed,Deaths,...,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax,Temp,Wind,Barometer,Visibility,Weather
0,Balochistan,Pakistan,2020-03-10,10,3,2020,28.328492,65.898403,1.0,0.0,...,0.0,0.0,25.615120,44.61881,10.791250,46.29,15.00,31.43,0.00,Haze.
1,Islamabad,Pakistan,2020-03-10,10,3,2020,33.665087,73.121219,2.0,0.0,...,0.0,0.0,0.004133,0.00000,0.000000,65.14,9.57,34.71,21.33,Clear.
2,Punjab,Pakistan,2020-03-10,10,3,2020,30.811346,72.139132,0.0,0.0,...,0.0,0.0,40.904370,89.53674,11.384460,56.40,2.44,23.67,11.88,Clear.
3,Sindh,Pakistan,2020-03-10,10,3,2020,26.009446,68.776807,0.0,0.0,...,0.0,0.0,22.339800,81.02134,6.179772,65.14,9.57,34.71,21.33,Clear.
4,Khyber Pakhtunkhwa,Pakistan,2020-03-10,10,3,2020,34.485332,72.091690,0.0,0.0,...,0.0,0.0,17.376130,39.58196,5.953569,48.80,4.64,49.20,23.21,Fog.
5,Azad Jammu and Kashmir,Pakistan,2020-03-10,10,3,2020,34.027401,73.947253,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,39.86,1.86,42.43,0.00,Clear.
6,Gilgit-Baltistan,Pakistan,2020-03-10,10,3,2020,35.792146,74.982138,2.0,0.0,...,0.0,0.0,12.495310,23.41426,3.297438,32.71,3.71,23.57,12.82,Fog.


# Defining Functions For Tensor Creation (Tabular to Image)

In [5]:
def createTensors(col):
    
    sindh = df[df['Province'] == 'Sindh'][col].values
    balochistan = df[df['Province'] == 'Balochistan'][col].values
    gilgit = df[df['Province'] == 'Gilgit-Baltistan'][col].values
    islamabad = df[df['Province'] == 'Islamabad'][col].values
    punjab = df[df['Province'] == 'Punjab'][col].values
    kpk = df[df['Province'] == 'Khyber Pakhtunkhwa'][col].values
    kashmir = df[df['Province'] == 'Azad Jammu and Kashmir'][col].values
    
    sb = np.zeros(len(sindh)*2)
    gi = np.zeros(len(sindh)*2)
    pk = np.zeros(len(sindh)*2)
    ax = np.zeros(len(sindh)*2)
    
    for i in range(len(sb)):
        if(i%2 == 0):
            sb[i] = sindh[i//2]
        elif(i%2 == 1):
            sb[i] = balochistan[(i - 1)//2]
        
    for i in range(len(gi)):
        if(i%2 == 0):
            gi[i] = gilgit[i//2]
        elif(i%2 == 1):
            gi[i] = islamabad[(i - 1)//2]
        
    for i in range(len(pk)):
        if(i%2 == 0):
            pk[i] = punjab[i//2]
        elif(i%2 == 1):
            pk[i] = kpk[(i - 1)//2]
    
    for i in range(len(ax)):
        if(i%2 == 0):
            ax[i] = kashmir[i//2]
        elif(i%2 == 1):
            ax[i] = -1
    
    AllData = np.zeros((4, len(sb)))
    AllData[0] = sb
    AllData[1] = gi
    AllData[2] = pk
    AllData[3] = ax
    
    
    tempArr = np.zeros((4, 8))
    tensorsArr = np.zeros((175, 4, 8))
    
    stIndex = 0
    enIndex = 8
    for tIndex in range(175):
        for i in range(4):
            tempArr[i] = AllData[i][stIndex:enIndex]
        stIndex += 8
        enIndex += 8
        tensorsArr[tIndex] = tempArr
    
    finalTensorsArr = np.zeros((88, 8, 8))
    stIndex = 0
    enIndex = 8
    for tIndex in range(0, 176, 2):
        tfIndex = tIndex//2
        for i in range(8):
            if(i < 4):
                finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i]
            else:
                finalTensorsArr[tfIndex][i] = tensorsArr[tIndex][i-4]

    convTensors = []
    
    for i in range(88):
        convTensors.append(tf.convert_to_tensor(finalTensorsArr[i], dtype=tf.float64))

    return convTensors

In [6]:
def createYtensor(col):
    y_temp = []
    for i in range(0, len(df['Date']), 7):
        try:
            y_temp.append(df[df['Date'] == str(datetime.datetime.strptime(df['Date'].iloc[i], '%Y-%m-%d') + datetime.timedelta(days=7))[:10]][col].values)
        except:
            continue
            
    y_temp = y_temp[:696]
    
    bData = []
    iData = []
    pData = []
    sData = []
    kData = []
    aData = []
    gData = []
    
    for i in range(len(y_temp)):
        try:
            bData.append(y_temp[i][0])
            iData.append(y_temp[i][1])
            pData.append(y_temp[i][2])
            sData.append(y_temp[i][3])
            kData.append(y_temp[i][4])
            aData.append(y_temp[i][5])
            gData.append(y_temp[i][6])
        except:
            continue
        
    bSum = np.zeros(100)
    iSum = np.zeros(100)
    pSum = np.zeros(100)
    sSum = np.zeros(100)
    kSum = np.zeros(100)
    aSum = np.zeros(100)
    gSum = np.zeros(100)

    for i in range(0, len(bData) - 7, 7): 
        try:
            for j in range(7):
                if (i > 0):
    #                 print(int(i//7))
                    bSum[int(i/7)] += bData[i+j]
                    iSum[int(i/7)] += iData[i+j]
                    pSum[int(i/7)] += pData[i+j]
                    sSum[int(i/7)] += sData[i+j]
                    kSum[int(i/7)] += kData[i+j]
                    aSum[int(i/7)] += aData[i+j]
                    gSum[int(i/7)] += gData[i+j]
                else:
    #                 print(int(i))
                    bSum[int(i)] += bData[i+j]
                    iSum[int(i)] += iData[i+j]
                    pSum[int(i)] += pData[i+j]
                    sSum[int(i)] += sData[i+j]
                    kSum[int(i)] += kData[i+j]
                    aSum[int(i)] += aData[i+j]
                    gSum[int(i)] += gData[i+j]
        except:
            continue

    ySum = np.zeros((100, 7, 1))
    
    for i in range(100):
        ySum[i][0] = bSum[i] 
        ySum[i][1] = iSum[i]  
        ySum[i][2] = pSum[i] 
        ySum[i][3] = sSum[i]  
        ySum[i][4] = kSum[i] 
        ySum[i][5] = aSum[i] 
        ySum[i][6] = gSum[i] 

    yTensors = np.zeros((100, 7, 1))
    for i in range(100):
        yTensors[i] = tf.convert_to_tensor(ySum[i], dtype=tf.float64)
    
    return yTensors[:88]
    

# Defining Function For Model Creation

In [7]:
def createModel():
    model = Sequential()

    model.add(Conv2D(8, (2, 2), input_shape=(8, 8, 6)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2,2), padding='same'))
    # model.add(Dropout(0.25))

    model.add(Conv2D(4, (2, 2)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2,2), padding='same'))
    # model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(36, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(7, activation='linear'))

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam')
    
    return model

# Data Preparation

In [8]:
comTensors = expand_dims(createTensors("Daily_Confirmed"), axis=3)
deathTensors = expand_dims(createTensors("Daily_Deaths"), axis=3)
recTensors = expand_dims(createTensors("Daily_Confirmed"), axis=3)
mobTensors = expand_dims(createTensors("mobility_mean"), axis=3)
popTensors = expand_dims(createTensors("population"), axis=3)
maskTensors = expand_dims(createTensors("mask_use_mean"), axis=3)

In [9]:
X = tf.concat([comTensors, deathTensors, recTensors, mobTensors, popTensors, maskTensors], 3)

# Model Implementation for Predicting Confirmed Daily Cases for each Province in next 7 days

In [10]:
y = createYtensor("Daily_Confirmed")[:88]

In [11]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [12]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [13]:
conModel = createModel()
conModel.fit(X_train, y_train, batch_size = 4, epochs = 50, validation_split = 0.2)

Epoch 1/50
14/14 [==============================] - 1s 17ms/step - loss: 53666648162304.0000 - val_loss: 15249475371008.0000
Epoch 2/50
14/14 [==============================] - 0s 4ms/step - loss: 8672284508160.0000 - val_loss: 4224250806272.0000
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 2330728857600.0000 - val_loss: 984975671296.0000
Epoch 4/50
14/14 [==============================] - 0s 3ms/step - loss: 542229725184.0000 - val_loss: 233925689344.0000
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 149098217472.0000 - val_loss: 73282207744.0000
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 42972139520.0000 - val_loss: 17806649344.0000
Epoch 7/50
14/14 [==============================] - 0s 4ms/step - loss: 9033881600.0000 - val_loss: 3142715392.0000
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 1551084288.0000 - val_loss: 521863680.0000
Epoch 9/50
14/14 [==========================

In [14]:
prediction = conModel.predict(X_test)

In [15]:
prediction[-1]

array([  16.3429 , 1476.5193 , 4256.1543 , 3250.5264 ,  893.49725,
        871.1695 , -142.20673], dtype=float32)

In [16]:
y_test[-1]

array([[  44.],
       [ 144.],
       [ 576.],
       [1044.],
       [ 482.],
       [  24.],
       [   5.]])

# Model Implementation for Predicting Confirmed Daily Deaths for each Province in next 7 days

In [17]:
y = createYtensor("Daily_Deaths")[:88]

In [18]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [19]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [20]:
deathModel = createModel()
deathModel.fit(X_train, y_train, batch_size = 4, epochs = 50, validation_split = 0.2)

Epoch 1/50
14/14 [==============================] - 1s 17ms/step - loss: 23963570798592.0000 - val_loss: 2786468560896.0000
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 903364149248.0000 - val_loss: 263015186432.0000
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 177816354816.0000 - val_loss: 68070711296.0000
Epoch 4/50
14/14 [==============================] - 0s 3ms/step - loss: 34919620608.0000 - val_loss: 25385189376.0000
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 21753853952.0000 - val_loss: 14736279552.0000
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 11003673600.0000 - val_loss: 8489182208.0000
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 6915673088.0000 - val_loss: 5224450560.0000
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 4288526848.0000 - val_loss: 3383159040.0000
Epoch 9/50
14/14 [==============================] - 

In [21]:
prediction = deathModel.predict(X_test)

In [22]:
prediction[-1]

array([61.121704 , -2.3676796, 22.486385 , 13.840597 ,  3.9235687,
       64.25782  , 49.686123 ], dtype=float32)

In [23]:
y_test[-1]

array([[ 0.],
       [ 4.],
       [21.],
       [ 8.],
       [18.],
       [ 0.],
       [ 0.]])

# Model Implementation for Predicting Confirmed Daily Recovered Cases for each Province in next 7 days

In [28]:
y = createYtensor("Recovered")[:88]

In [29]:
# Giving 80% data for Training
X_train = X[:70]
y_train = y[:70]

In [30]:
# Giving 20% data for Training
X_test = X[70:]
y_test = y[70:]

In [31]:
recModel = createModel()
recModel.fit(X_train, y_train, batch_size = 4, epochs = 50, validation_split = 0.2)

Epoch 1/50
14/14 [==============================] - 0s 11ms/step - loss: 3057085841408.0000 - val_loss: 966949928960.0000
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 341661089792.0000 - val_loss: 720839245824.0000
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 121760169984.0000 - val_loss: 523520245760.0000
Epoch 4/50
14/14 [==============================] - 0s 4ms/step - loss: 87799087104.0000 - val_loss: 574400888832.0000
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 81261625344.0000 - val_loss: 577326350336.0000
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 79833128960.0000 - val_loss: 574868226048.0000
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 78929707008.0000 - val_loss: 580428169216.0000
Epoch 8/50
14/14 [==============================] - 0s 4ms/step - loss: 78693457920.0000 - val_loss: 574693703680.0000
Epoch 9/50
14/14 [=========================

In [32]:
prediction = recModel.predict(X_test)

In [33]:
prediction[-1]

array([ 82450.11  ,  93733.26  , 629762.7   , 810793.4   , 204496.27  ,
        34246.21  ,  -1954.6055], dtype=float32)

In [34]:
y_test[-1]

array([[ 223690.],
       [ 701595.],
       [2814007.],
       [3040854.],
       [1172976.],
       [ 218234.],
       [  71301.]])